# Kaggle Future Sales Prediction

## 01: Framing the problem


https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data

From the given 2 years of sales data, find the count of sale for each item for each shop for the month of November 2015

For a given shop id and item id predict the count of sale for November 2015

## 02 : Obtain Data

#### Importing the basic required libraries

In [0]:
import pandas as pd
import numpy as np
from math import ceil
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as ms
%matplotlib inline


#### Reading the data from CSV file

In [0]:
!wget -q https://www.dropbox.com/s/fuykntsynlcyjug/items.csv


In [0]:
items = pd.read_csv("items.csv")

In [65]:
items.columns

Index(['item_name', 'item_id', 'item_category_id'], dtype='object')

In [67]:
items.head(5)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


##03 : Analyze Data

In [116]:
items.isnull().sum()

item_name           0
item_id             0
item_category_id    0
dtype: int64

##04 : Feature Engineering

shops.columns == 'shop_id', 'shop_name', <br/>

item_categories == 'item_category_id', 'item_category_name',  <br/>

items == 'item_id', item_name','item_category_id'   <br/>

sales_train == 'date', 'date_block_num', 'shop_id', 'item_id', 'item_price','item_cnt_day'<br/>
test == 'ID', 'shop_id', 'item_id' <br/>
sample_submission == 'ID', 'item_cnt_month'<br/>

In [0]:
pd.options.display.float_format = '{:.3f}'.format

In [0]:
!wget -q https://www.dropbox.com/s/591ljx20pv163ib/sales_train.csv

In [0]:
sales_train = pd.read_csv('sales_train.csv')

In [71]:
sales_train.shape

(2935849, 6)

In [0]:
sales_train['Year'] = sales_train['date'].apply(lambda x:int(x.split(".")[2]))
sales_train['Month'] = sales_train['date'].apply(lambda x:int(x.split(".")[1]))

In [73]:
sales_train.query("shop_id==5 and item_id==5037")

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,Year,Month
1953995,21.09.2014,20,5,5037,2599.000,1.000,2014,9
2150561,29.11.2014,22,5,5037,2599.000,1.000,2014,11
2288630,28.12.2014,23,5,5037,1999.000,1.000,2014,12
2288631,20.12.2014,23,5,5037,1999.000,1.000,2014,12
2335446,02.01.2015,24,5,5037,1999.000,1.000,2015,1
2335447,07.01.2015,24,5,5037,1999.000,1.000,2015,1
2618926,29.05.2015,28,5,5037,1299.000,1.000,2015,5
2704068,28.06.2015,29,5,5037,1499.000,1.000,2015,6
2719247,05.07.2015,30,5,5037,1499.000,1.000,2015,7
2810661,14.08.2015,31,5,5037,1499.000,1.000,2015,8


In [0]:
#df.query("shop_id==5 and item_id==5037")

In [0]:
allFeatures = sales_train.drop(["date",  "item_price"], axis=1)

In [76]:
allFeatures.head(5)

,date_block_num,shop_id,item_id,item_cnt_day,Year,Month
0,0,59,22154,1.000,2013,1
1,0,25,2552,1.000,2013,1
2,0,25,2552,-1.000,2013,1
3,0,25,2554,1.000,2013,1
4,0,25,2555,1.000,2013,1


In [77]:
df = allFeatures.groupby(["Year", "Month", "shop_id", "item_id"]).sum().reset_index()
df.rename(columns={'item_cnt_day':'item_cnt_month'},inplace = True)
df.shape

(1609124, 6)

In [78]:
df.head(5)

,Year,Month,shop_id,item_id,date_block_num,item_cnt_month
0,2013,1,0,32,0,6.000
1,2013,1,0,33,0,3.000
2,2013,1,0,35,0,1.000
3,2013,1,0,43,0,1.000
4,2013,1,0,51,0,2.000


In [79]:
X_train = df.query("not (Year == 2015 and Month == 10)").drop(["item_cnt_month"],axis=1)
y_train = df.query("not (Year == 2015 and Month == 10)")["item_cnt_month"]
X_train.shape

(1577593, 5)

In [80]:
X_test = df.query("Year == 2015 and Month == 10").drop(["item_cnt_month"],axis=1)
y_test = df.query("Year == 2015 and Month == 10")["item_cnt_month"]
X_test.shape

(31531, 5)

In [81]:
1577593 + 31531 == 1609124

True

##05 : Model Selection 

#######Year Month shop_id item_id date_block_num item_price item_cnt_day

In [0]:
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import sklearn.metrics as metrics
import math


In [0]:
regr_model = RandomForestRegressor(max_depth=10, random_state=0,  n_estimators=100)

In [84]:
X_train.head(5)

,Year,Month,shop_id,item_id,date_block_num
0,2013,1,0,32,0
1,2013,1,0,33,0
2,2013,1,0,35,0
3,2013,1,0,43,0
4,2013,1,0,51,0


In [85]:
regr_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

## 06 : Tune the Model

In [0]:
#RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
#                      max_features='auto', max_leaf_nodes=None,
#                      min_impurity_decrease=0.0, min_impurity_split=None,
#                     min_samples_leaf=1, min_samples_split=2,
#                      min_weight_fraction_leaf=0.0, n_estimators=10,
#                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
#                      warm_start=False)

##07 : Predict on new cases

########## predict with train dataset itself

In [0]:
# Show model results of training set
#https://www.kaggle.com/rogerbellavista/randomforestregressor-mae-0-0922-rmse-0-2314
def eval(model,X,y):
  y_train_hat = model.predict(X)
  mae = metrics.mean_absolute_error(y,y_train_hat)
  mse = metrics.mean_squared_error(y,y_train_hat)
  print ("TRAINING SET")
  print ("============")
  print ("MAE:                ", mae)
  print ("RMSE:               ", math.sqrt(mse))
  
  
  #print ("last column (% Iron Concentrate) is the highest feature_importances")

In [87]:
eval(regr_model,X_train, y_train)

TRAINING SET
MAE:                 0.4374999783696802
RMSE:                3.4560393632250404


In [88]:
eval(regr_model,X_test, y_test)

TRAINING SET
MAE:                 1.5537707896576591
RMSE:                14.244745695686946


## Kaggle Submission

In [0]:
!wget -q  https://www.dropbox.com/s/2go24xg61frrrod/test.csv

In [90]:
tests = pd.read_csv("test.csv")
tests.head(5)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [91]:
tests.shape

(214200, 3)

In [92]:
newtest = tests.copy()
newtest.head(5)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [93]:
newtest["Year"] = 2015
newtest.head(5)

,ID,shop_id,item_id,Year
0,0,5,5037,2015
1,1,5,5320,2015
2,2,5,5233,2015
3,3,5,5232,2015
4,4,5,5268,2015


In [94]:
newtest["Month"] = 11
newtest.head(5)


,ID,shop_id,item_id,Year,Month
0,0,5,5037,2015,11
1,1,5,5320,2015,11
2,2,5,5233,2015,11
3,3,5,5232,2015,11
4,4,5,5268,2015,11


In [95]:
newtest["date_block_num"] = 33
newtest.head(5)

,ID,shop_id,item_id,Year,Month,date_block_num
0,0,5,5037,2015,11,33
1,1,5,5320,2015,11,33
2,2,5,5233,2015,11,33
3,3,5,5232,2015,11,33
4,4,5,5268,2015,11,33


In [96]:
x_cols = X_train.columns
x_cols

Index(['Year', 'Month', 'shop_id', 'item_id', 'date_block_num'], dtype='object')

In [0]:
newtest.drop("ID", axis = 1, inplace = True)



In [98]:
newtest.head(5)

,shop_id,item_id,Year,Month,date_block_num
0,5,5037,2015,11,33
1,5,5320,2015,11,33
2,5,5233,2015,11,33
3,5,5232,2015,11,33
4,5,5268,2015,11,33


In [99]:
kaggle_test = newtest[x_cols]
kaggle_test.head(5)

,Year,Month,shop_id,item_id,date_block_num
0,2015,11,5,5037,33
1,2015,11,5,5320,33
2,2015,11,5,5233,33
3,2015,11,5,5232,33
4,2015,11,5,5268,33


In [0]:
new_y_hat = regr_model.predict(kaggle_test)


In [0]:
type(new_y_hat)
df_sub = pd.DataFrame(new_y_hat)

In [102]:
new_y_hat

array([2.0334654 , 2.0334654 , 2.0334654 , ..., 1.94961498, 1.94225848,
       2.04522814])

In [103]:
df_submission = pd.DataFrame(new_y_hat, columns=['item_cnt_month'])

df_submission.head(5)

,item_cnt_month
0,2.033
1,2.033
2,2.033
3,2.033
4,2.033


In [0]:
df_submission.to_csv("temp.csv",index=True,index_label="ID")

In [105]:
pwd

'/content'

In [0]:
!wget -q  https://www.dropbox.com/s/2ru4kmnqiqggv6l/sample_submission.csv

In [107]:
sampleSubmission = pd.read_csv("sample_submission.csv")
sampleSubmission.head(5)

,ID,item_cnt_month
0,0,0.500
1,1,0.500
2,2,0.500
3,3,0.500
4,4,0.500


In [108]:
type(sampleSubmission)

pandas.core.frame.DataFrame

In [109]:
df_submission['ID'] = sampleSubmission['ID']
df_submission.head(5)


,item_cnt_month,ID
0,2.033,0
1,2.033,1
2,2.033,2
3,2.033,3
4,2.033,4


In [110]:
final_submission = df_submission[['ID', 'item_cnt_month']]
final_submission.head(5)

,ID,item_cnt_month
0,0,2.033
1,1,2.033
2,2,2.033
3,3,2.033
4,4,2.033


In [0]:
final_submission.to_csv("futurePredictSales_Nov2015.csv", index=False)

In [112]:
ls -al

total 208972
drwxr-xr-x 1 root root     4096 Nov 22 05:05 ./
drwxr-xr-x 1 root root     4096 Nov 22 03:13 ../
drwxr-xr-x 1 root root     4096 Nov 20 16:17 .config/
-rw-r--r-- 1 root root  5369878 Nov 22 05:06 futurePredictSales_Nov2015.csv
-rw-r--r-- 1 root root  1568417 Nov 22 03:16 items.csv
-rw-r--r-- 1 root root  1568417 Nov 22 05:00 items.csv.1
-rw-r--r-- 1 root root 94603866 Nov 22 03:26 sales_train.csv
-rw-r--r-- 1 root root 94603866 Nov 22 05:00 sales_train.csv.1
drwxr-xr-x 1 root root     4096 Nov 15 16:31 sample_data/
-rw-r--r-- 1 root root  2245108 Nov 22 04:31 sample_submission.csv
-rw-r--r-- 1 root root  2245108 Nov 22 05:05 sample_submission.csv.1
-rw-r--r-- 1 root root  5369878 Nov 22 05:05 temp.csv
-rw-r--r-- 1 root root  3182735 Nov 22 04:29 test.csv
-rw-r--r-- 1 root root  3182735 Nov 22 05:05 test.csv.1


In [0]:
from google.colab import files

files.download('temp.csv')

In [115]:
final_submission.shape

(214200, 2)

In [0]:
#df.index = df['value']

#df.reindex(np.arange(df.value.min(), df.value.max() + 1)).fillna(0)

#df.reindex(np.arange(df.value.min(), df.value.max() + 1), fill_value=0)